In [1]:
import os
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model

LOGDIR = "./save"

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = (
    tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y)))
    + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version=saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = "./logs"
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

epochs = 1
batch_size = 50

# train over the dataset about 30 times
for epoch in range(epochs):
    for i in range(int(driving_data.num_images / batch_size)):
        xs, ys = driving_data.LoadTrainBatch(batch_size)
        train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.8})
        if i % 10 == 0:
            xs, ys = driving_data.LoadValBatch(batch_size)
            loss_value = loss.eval(
                feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}
            )
            print(
                "Epoch: %d, Step: %d, Loss: %g"
                % (epoch, epoch * batch_size + i, loss_value)
            )

        # write logs at every iteration
        summary = merged_summary_op.eval(
            feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 1.0}
        )
        summary_writer.add_summary(
            summary, epoch * driving_data.num_images / batch_size + i
        )

        if i % batch_size == 0:
            if not os.path.exists(LOGDIR):
                os.makedirs(LOGDIR)
            checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
            filename = saver.save(sess, checkpoint_path)
    print("Model saved in file: %s" % filename)

print("Run the command line:\n"
      "--> tensorboard --logdir=./logs "
      "\nThen open http://0.0.0.0:6006/ into your web browser")

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch: 0, Step: 0, Loss: 6.9573
Epoch: 0, Step: 10, Loss: 6.24655
Epoch: 0, Step: 20, Loss: 6.18404
Epoch: 0, Step: 30, Loss: 6.26051
Epoch: 0, Step: 40, Loss: 6.49872
Epoch: 0, Step: 50, Loss: 6.21913
Epoch: 0, Step: 60, Loss: 6.14615
Epoch: 0, Step: 70, Loss: 6.03394
Epoch: 0, Step: 80, Loss: 5.98904
Epoch: 0, Step: 90, Loss: 5.96299
Epoch: 0, Step: 100, Loss: 6.03691
Epoch: 0, Step: 110, Loss: 5.78491
Epoch: 0, Step: 120, Loss: 5.77361
Epoch: 0, Step: 130, Loss: 5.8131
Epoch: 0, Step: 140, Loss: 6.07596
Epoch: 0, Step: 150, Loss: 5.71242
Epoch: 0, Step: 160, Loss: 5.53873
Epoch: 0, Step: 170, Loss: 5.43322
Epoch: 0, Step: 180, Loss: 5.52739
Epoch: 0, Step: 190, Loss: 5.38921
Epoch: 0, Step: 200, Loss: 5.5999
Epoch: 0, Step: 210, Loss: 5.97905
Epoch: 0, Step: 220, Loss: 5.37254
Epoch: 0,

In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import model
import cv2
from subprocess import call
import os

# check if on windows OS
windows = False
if os.name == "nt":
    windows = True

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread("steering_wheel_image.jpg", 0)
rows, cols = img.shape

smoothed_angle = 0

i = 0
while cv2.waitKey(10) != ord("q"):
    full_image = cv2.imread("driving_dataset/" + str(i) + ".jpg")
    image = cv2.resize(full_image[-150:], (200, 66)) / 255.0
    degrees = (
        model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0]
        * 180.0
        / 3.14159265
    )
    if not windows:
        call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow("frame", full_image)
    # make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    # and the predicted angle
    smoothed_angle += (
        0.2
        * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0)
        * (degrees - smoothed_angle)
        / abs(degrees - smoothed_angle)
    )
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), -smoothed_angle, 1)
    dst = cv2.warpAffine(img, M, (cols, rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from save/model.ckpt


C:\Users\tmayu\Desktop\Coding\ML\virtual stuff\mltensor\lib\site-packages\tensorflow\python\client\session.py:1768: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Predicted steering angle: -3.4171011704230856 degrees
Predicted steering angle: -6.267596740524329 degrees
Predicted steering angle: -7.373784297685979 degrees
Predicted steering angle: -7.7573122266251335 degrees
Predicted steering angle: -6.734621597728182 degrees
Predicted steering angle: -10.48981734589664 degrees
Predicted steering angle: -7.157350543679772 degrees
Predicted steering angle: -7.694872344708821 degrees
Predicted steering angle: -8.872172416495317 degrees
Predicted steering angle: -11.427208730359595 degrees
Predicted steering angle: -9.602897644363168 degrees
Predicted steering angle: -8.79984668951776 degrees
Predicted steering angle: -8.578555390345656 degrees
Predicted steering angle: -7.5934167146615215 degrees
Predicted steering angle: -7.157523859730159 degrees
Predicted steering angle: -4.717042624982588 degrees
Predicted steering angle: -6.152253201444409 degrees
Predicted steering angle: -5.6334158097981035 degrees
Predicted steering angle: -4.8732951552562

Predicted steering angle: -2.1641467216519077 degrees
Predicted steering angle: -6.355988779995405 degrees
Predicted steering angle: -1.9226382759568748 degrees
Predicted steering angle: -2.055834435443727 degrees
Predicted steering angle: 2.4321486173940468 degrees
Predicted steering angle: 2.5690187783283296 degrees
Predicted steering angle: 3.4978585524972448 degrees
Predicted steering angle: 1.1700501461393602 degrees
Predicted steering angle: -0.08620978068855051 degrees
Predicted steering angle: -1.5686077996425956 degrees
Predicted steering angle: 2.533529328803802 degrees
Predicted steering angle: 5.422790277913983 degrees
Predicted steering angle: 6.543090713464299 degrees
Predicted steering angle: 5.741623935621322 degrees
Predicted steering angle: 6.296072226093473 degrees
Predicted steering angle: 7.453972426180849 degrees
Predicted steering angle: 7.359138664541444 degrees
Predicted steering angle: 6.629078953231607 degrees
Predicted steering angle: 5.9085430869565885 degr

Predicted steering angle: 3.211843526902495 degrees
Predicted steering angle: 3.4061024314126813 degrees
Predicted steering angle: 2.102558478948705 degrees
Predicted steering angle: 1.6592699165199432 degrees
Predicted steering angle: 0.8331531460167829 degrees
Predicted steering angle: -0.6932088129831002 degrees
Predicted steering angle: -1.287094188881145 degrees
Predicted steering angle: -0.9662584319710583 degrees
Predicted steering angle: -1.1896788291759364 degrees
Predicted steering angle: -0.40235198367407193 degrees
Predicted steering angle: -1.7975301276736173 degrees
Predicted steering angle: -0.1957479124313774 degrees
Predicted steering angle: -1.0600040590970727 degrees
Predicted steering angle: -2.65457842372473 degrees
Predicted steering angle: -1.0884261838132654 degrees
Predicted steering angle: 0.48409531422821966 degrees
Predicted steering angle: 0.7034451321265304 degrees
Predicted steering angle: -0.8394878263140909 degrees
Predicted steering angle: 0.5456861716

...

Predicted steering angle: -9.278493540494495 degrees
Predicted steering angle: -9.56430963682944 degrees
Predicted steering angle: -12.172829747050619 degrees
Predicted steering angle: -13.852523530037823 degrees
Predicted steering angle: -10.365834898167252 degrees
Predicted steering angle: -14.837392413249434 degrees
Predicted steering angle: -8.30987111601597 degrees
Predicted steering angle: -5.816020063693403 degrees
Predicted steering angle: -7.653758020952949 degrees
Predicted steering angle: -6.03079749682399 degrees
Predicted steering angle: -6.175772957878223 degrees
Predicted steering angle: -4.90459108206898 degrees
Predicted steering angle: -3.042003829363476 degrees
Predicted steering angle: 0.4849473803281524 degrees
Predicted steering angle: -2.614913380735026 degrees
Predicted steering angle: 2.71217698217444 degrees
Predicted steering angle: 1.466842143039647 degrees
Predicted steering angle: 2.1605211718589574 degrees
Predicted steering angle: -7.051858271828383 degr

Predicted steering angle: -9.06941632711962 degrees
Predicted steering angle: -5.162288997106146 degrees
Predicted steering angle: -2.6093087836278035 degrees
Predicted steering angle: 3.192660513709771 degrees
Predicted steering angle: 0.6259293685083339 degrees
Predicted steering angle: 1.238131123986314 degrees
Predicted steering angle: -1.8350090832395958 degrees
Predicted steering angle: 1.4460151886891872 degrees
Predicted steering angle: 1.9091719176625723 degrees
Predicted steering angle: 4.311787437381972 degrees
Predicted steering angle: 4.54574062662928 degrees
Predicted steering angle: 0.24075784525698787 degrees
Predicted steering angle: 3.5560726817466057 degrees
Predicted steering angle: 4.498761304803781 degrees
Predicted steering angle: 4.374179081087216 degrees
Predicted steering angle: 2.77250676252163 degrees
Predicted steering angle: -1.2220435738733182 degrees
Predicted steering angle: -3.615838117712979 degrees
Predicted steering angle: -6.562931559252211 degrees

Predicted steering angle: -11.249660524003174 degrees
Predicted steering angle: -11.723208638758667 degrees
Predicted steering angle: -11.104635117190577 degrees
Predicted steering angle: -10.55754806310482 degrees
Predicted steering angle: -9.367514811143577 degrees
Predicted steering angle: -10.058324413456264 degrees
Predicted steering angle: -9.359472263386698 degrees
Predicted steering angle: -9.66003815325114 degrees
Predicted steering angle: -9.24936107610381 degrees
Predicted steering angle: -8.876424209258508 degrees
Predicted steering angle: -8.07689336316062 degrees
Predicted steering angle: -9.300251961894078 degrees
Predicted steering angle: -7.122287767535447 degrees
Predicted steering angle: -7.161937869486324 degrees
Predicted steering angle: -7.432074839676767 degrees
Predicted steering angle: -7.81224402308773 degrees
Predicted steering angle: -7.444045599984539 degrees
Predicted steering angle: -8.84125471140828 degrees
Predicted steering angle: -7.243956061794026 de

Predicted steering angle: -7.748118791991791 degrees
Predicted steering angle: -6.746213282536586 degrees
Predicted steering angle: -7.886372066234581 degrees
Predicted steering angle: -8.201879848699146 degrees
Predicted steering angle: -7.143251752556287 degrees
Predicted steering angle: -7.213291073706568 degrees
Predicted steering angle: -7.160641841089833 degrees
Predicted steering angle: -7.726347563987498 degrees
Predicted steering angle: -8.416648317206437 degrees
Predicted steering angle: -8.111156153397493 degrees
Predicted steering angle: -7.191170652280559 degrees
Predicted steering angle: -6.976170384228047 degrees
Predicted steering angle: -7.539713498477285 degrees
Predicted steering angle: -6.954750057192765 degrees
Predicted steering angle: -7.806400796246106 degrees
Predicted steering angle: -7.155889310082543 degrees
Predicted steering angle: -6.842958643372133 degrees
Predicted steering angle: -7.790381441303183 degrees
Predicted steering angle: -7.229673282449562 d

Predicted steering angle: 0.0018535425881301914 degrees
Predicted steering angle: 0.7464147060178268 degrees
Predicted steering angle: -6.802389454090016 degrees
Predicted steering angle: -9.860347965042639 degrees
Predicted steering angle: -5.512179951891845 degrees
Predicted steering angle: -8.755159323048243 degrees
Predicted steering angle: -9.799795776660345 degrees
Predicted steering angle: -9.336695290025972 degrees
Predicted steering angle: -8.705814621333102 degrees
Predicted steering angle: -7.358953395659996 degrees
Predicted steering angle: -9.06616174197639 degrees
Predicted steering angle: -8.613090534375996 degrees
Predicted steering angle: -8.788483816046815 degrees
Predicted steering angle: -10.899658578643336 degrees
Predicted steering angle: -10.55430543079265 degrees
Predicted steering angle: -12.26943850410261 degrees
Predicted steering angle: -16.045570916535194 degrees
Predicted steering angle: -14.966262715109197 degrees
Predicted steering angle: -16.27353360298

In [ ]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import model
import cv2
from subprocess import call
import os

# check if on windows OS
windows = False
if os.name == "nt":
    windows = True

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, "save/model.ckpt")

img = cv2.imread("steering_wheel_image.jpg", 0)
rows, cols = img.shape

smoothed_angle = 0

i = 0
while cv2.waitKey(10) != ord("q"):
    full_image = cv2.imread("driving_dataset/" + str(i) + ".jpg")
    image = cv2.resize(full_image[-150:], (200, 66)) / 255.0
    degrees = (
        model.y.eval(feed_dict={model.x: [image], model.keep_prob: 1.0})[0][0]
        * 180.0
        / 3.14159265
    )
    if not windows:
        call("clear")
    print("Predicted steering angle: " + str(degrees) + " degrees")
    cv2.imshow("frame", full_image)
    # make smooth angle transitions by turning the steering wheel based on the difference of the current angle
    # and the predicted angle
    smoothed_angle += (
        0.2
        * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0)
        * (degrees - smoothed_angle)
        / abs(degrees - smoothed_angle)
    )
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), -smoothed_angle, 1)
    dst = cv2.warpAffine(img, M, (cols, rows))
    cv2.imshow("steering wheel", dst)
    i += 1

cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from save/model.ckpt


C:\Users\manis\anaconda3\lib\site-packages\tensorflow\python\client\session.py:1769: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Predicted steering angle: 2.1420171221591913 degrees
Predicted steering angle: 0.7348550910820474 degrees
Predicted steering angle: 0.1451294871562787 degrees
Predicted steering angle: 1.7036455488865168 degrees
Predicted steering angle: 0.34005705651480317 degrees
Predicted steering angle: 0.16249692400774737 degrees
Predicted steering angle: -1.088414230982204 degrees
Predicted steering angle: 0.7512673419852541 degrees
Predicted steering angle: 1.9531049751147647 degrees
Predicted steering angle: 1.847390722115096 degrees
Predicted steering angle: 1.5795613953714025 degrees
Predicted steering angle: 0.013867845351910962 degrees
Predicted steering angle: -1.2276880848985487 degrees
Predicted steering angle: -2.2516901101176403 degrees
Predicted steering angle: -1.0465692904277177 degrees
Predicted steering angle: -1.4782424758293655 degrees
Predicted steering angle: -1.4209198997111212 degrees
Predicted steering angle: -1.2220760172719127 degrees
Predicted steering angle: -1.66569072

Predicted steering angle: -0.433023401744311 degrees
Predicted steering angle: 0.48547671998943326 degrees
Predicted steering angle: 2.24750576547258 degrees
Predicted steering angle: 1.3074540223248654 degrees
Predicted steering angle: 1.2423115199282557 degrees
Predicted steering angle: 1.8429700956131656 degrees
Predicted steering angle: 2.42061200098589 degrees
Predicted steering angle: 1.4817785928327685 degrees
Predicted steering angle: 1.5541748628579262 degrees
Predicted steering angle: 1.169501276405899 degrees
Predicted steering angle: 1.2732818388163039 degrees
Predicted steering angle: 2.3578545152728094 degrees
Predicted steering angle: 1.3274886747306982 degrees
Predicted steering angle: 2.842515561264023 degrees
Predicted steering angle: 2.38078559477686 degrees
Predicted steering angle: 3.450081379198412 degrees
Predicted steering angle: 2.4474898626176604 degrees
Predicted steering angle: 3.1805586991471495 degrees
Predicted steering angle: -0.22846099677682785 degrees

Predicted steering angle: 2.8354224098028595 degrees
Predicted steering angle: 2.0548957113186006 degrees
Predicted steering angle: 1.8655342658923206 degrees
Predicted steering angle: 2.4097891393468145 degrees
Predicted steering angle: 1.4802539665422327 degrees
Predicted steering angle: 1.4676723311898694 degrees
Predicted steering angle: 0.812667914568626 degrees
Predicted steering angle: 0.4537049210900508 degrees
Predicted steering angle: -0.39924342050494427 degrees
Predicted steering angle: -0.2334487157435025 degrees
Predicted steering angle: 0.9060859594183699 degrees
Predicted steering angle: 0.07930616697707159 degrees
Predicted steering angle: 0.40057189230002976 degrees
Predicted steering angle: -0.30019978105234113 degrees
Predicted steering angle: -0.35793488992475486 degrees
Predicted steering angle: 0.10011688628802068 degrees
Predicted steering angle: 0.43019402255782657 degrees
Predicted steering angle: 0.8481420495283561 degrees
Predicted steering angle: 0.62879628

...

Predicted steering angle: -0.40545030155943285 degrees
Predicted steering angle: 0.0019005001387276859 degrees
Predicted steering angle: -0.8645916530286292 degrees
Predicted steering angle: -0.14252206243764692 degrees
Predicted steering angle: 0.2805187243385927 degrees
Predicted steering angle: 0.8743590903571727 degrees
Predicted steering angle: 2.7117532970020037 degrees
Predicted steering angle: 2.4742936593855336 degrees
Predicted steering angle: 1.1697974291397808 degrees
Predicted steering angle: 1.1441309655917198 degrees
Predicted steering angle: 1.6889109098392585 degrees
Predicted steering angle: -0.9811941347122387 degrees
Predicted steering angle: -1.049424949834281 degrees
Predicted steering angle: 0.02378442626445457 degrees
Predicted steering angle: -0.9479761499761593 degrees
Predicted steering angle: -1.065244841908871 degrees
Predicted steering angle: -0.9137157076168155 degrees
Predicted steering angle: 1.0541514407453305 degrees
Predicted steering angle: 0.267995

Predicted steering angle: -2.224963579864841 degrees
Predicted steering angle: -2.317864611410336 degrees
Predicted steering angle: -1.7615019469776898 degrees
Predicted steering angle: -2.1818469434628094 degrees
Predicted steering angle: -1.4535467529182018 degrees
Predicted steering angle: -0.24298189892758817 degrees
Predicted steering angle: -0.8291670903012911 degrees
Predicted steering angle: -0.1582587916229117 degrees
Predicted steering angle: -0.9145557141639812 degrees
Predicted steering angle: -0.8253901557685169 degrees
Predicted steering angle: -0.42710341531571716 degrees
Predicted steering angle: -1.2484997780450642 degrees
Predicted steering angle: -0.7361408208339185 degrees
Predicted steering angle: -1.3384615870882737 degrees
Predicted steering angle: -2.24411414965897 degrees
Predicted steering angle: -2.132981422207174 degrees
Predicted steering angle: -2.509990149019514 degrees
Predicted steering angle: 0.5259140546039067 degrees
Predicted steering angle: -1.1112

Predicted steering angle: 1.4358944491532504 degrees
Predicted steering angle: 0.9102111802383598 degrees
Predicted steering angle: 0.01477455296526622 degrees
Predicted steering angle: 0.7658986744211981 degrees
Predicted steering angle: 0.05089685887074735 degrees
Predicted steering angle: 1.2554055261906611 degrees
Predicted steering angle: 1.3268223043990375 degrees
Predicted steering angle: 0.13727990558388534 degrees
Predicted steering angle: 1.9126954415046789 degrees
Predicted steering angle: -0.6337650895991731 degrees
Predicted steering angle: 0.34002375934256135 degrees
Predicted steering angle: -0.5213609864649501 degrees
Predicted steering angle: -0.2661967514476363 degrees
Predicted steering angle: -1.227782853773391 degrees
Predicted steering angle: -2.4889392925300684 degrees
Predicted steering angle: -1.6538103542110432 degrees
Predicted steering angle: -3.635457836126259 degrees
Predicted steering angle: -3.1222380616352967 degrees
Predicted steering angle: 0.22840977

Predicted steering angle: 0.8108075951521709 degrees
Predicted steering angle: 1.2281793249108222 degrees
Predicted steering angle: 1.3904268402918734 degrees
Predicted steering angle: 1.3835668757581088 degrees
Predicted steering angle: -0.3009442716727232 degrees
Predicted steering angle: -0.07368748260466956 degrees
Predicted steering angle: -1.6033506241126227 degrees
Predicted steering angle: -1.7589377445499488 degrees
Predicted steering angle: -2.0779717188992683 degrees
Predicted steering angle: -2.532501812219364 degrees
Predicted steering angle: -2.09089102172911 degrees
Predicted steering angle: -2.6284804843197014 degrees
Predicted steering angle: -1.943425956719562 degrees
Predicted steering angle: -3.467990562109473 degrees
Predicted steering angle: -2.9663919276177433 degrees
Predicted steering angle: -2.8993702694235863 degrees
Predicted steering angle: -0.19318146884781254 degrees
Predicted steering angle: -1.6426248522153082 degrees
Predicted steering angle: -0.902595

Predicted steering angle: 2.9985907200624453 degrees
Predicted steering angle: 3.8609249422069527 degrees
Predicted steering angle: 2.918818592777186 degrees
Predicted steering angle: 1.8986376315162599 degrees
Predicted steering angle: 2.0646238216985187 degrees
Predicted steering angle: 1.893499408263835 degrees
Predicted steering angle: 2.027495340214727 degrees
Predicted steering angle: 1.4109535864836267 degrees
Predicted steering angle: 2.8610612324290914 degrees
Predicted steering angle: 3.002956064720718 degrees
Predicted steering angle: 3.2234243260968958 degrees
Predicted steering angle: 1.2847551692528603 degrees
Predicted steering angle: 0.5079826202171651 degrees
Predicted steering angle: 0.34666689205035656 degrees
Predicted steering angle: 0.9030313707520029 degrees
Predicted steering angle: 0.2716497770520498 degrees
Predicted steering angle: -0.18709238187621482 degrees
Predicted steering angle: -0.5874167598598281 degrees
Predicted steering angle: -0.7540693203737479 